# Network Operations
## Workflow

In [3]:
import os

from kfp import dsl
from mlrun import new_project, new_function, mount_v3io, import_function, NewTask, mlconf, code_to_function

In [4]:
mlconf.dbpath = 'http://mlrun-api:8080'

## Define project

In [5]:
# update the dir and repo to reflect real locations 
# the remote git repo must be initialized in GitHub
project_dir = '/User/demo-network-operations'
remote_git = 'https://github.com/zilbermanor/demo-network-operations.git'
newproj = new_project('network-operations', project_dir, init_git=True)

MLRUN_COMMIT='v0.4.5'
ARTIFACT_PATH =  os.path.join(project_dir, 'artifacts', '{{run.uid}}')

In [6]:
newproj.pull()

## Setup functions

In [7]:
# Aggregate
newproj.set_function(import_function(os.path.join(project_dir, 'yaml', 'aggregate.yaml')), 'aggregate')

# Summary
newproj.set_function(code_to_function(name='describe',
                                      project='network-operations',
                                      filename='https://raw.githubusercontent.com/yjb-ds/lightgbm-project/pre-project/notebooks/code/describe.py',
                                      image=f'yjbds/mlrun-ml:{MLRUN_COMMIT}',
                                      kind='job').apply(mount_v3io()), 'describe')

print(newproj.to_yaml())

name: network-operations
functions:
- name: aggregate
  spec:
    kind: job
    metadata:
      name: aggregate
      tag: ''
      hash: 06454ac064876899b23bc9e3128f53c84b1c2ba2
      project: network-operations
    spec:
      command: /User/demo-network-operations/src/aggregate.py
      args: []
      image: ''
      env:
      - name: V3IO_API
        value: ''
      - name: V3IO_USERNAME
        value: ''
      - name: V3IO_ACCESS_KEY
        value: ''
      description: ''
      build:
        source: ./
        commands: []
        code_origin: '#0b8008a76e97b7806250ecf029094b597dd94507'
- name: describe
  spec:
    kind: job
    metadata:
      name: describe
      tag: ''
      project: network-operations
    spec:
      command: ''
      args: []
      image: yjbds/mlrun-ml:v0.4.5
      env:
      - name: V3IO_API
        value: ''
      - name: V3IO_USERNAME
        value: ''
      - name: V3IO_ACCESS_KEY
        value: ''
      description: ''
      build:
        functionS

### Test Summary

In [8]:
summ_task = NewTask(
    "sum", 
    handler="table_summary",  
    params={"key": "summary", "label_column": "is_error", 'class_labels': [0, 1]},
    inputs={"table": os.path.join(project_dir, 'data', 'aggregate.pq')},
    artifact_path=ARTIFACT_PATH)

In [9]:
newproj.func('describe').run(summ_task)

[mlrun] 2020-03-10 12:31:47,022 starting run sum uid=113baae308414b27ac6edc6034347bf8  -> http://mlrun-api:8080
[mlrun] 2020-03-10 12:31:47,095 Job is running in the background, pod: sum-24vdl
Intel(R) Data Analytics Acceleration Library (Intel(R) DAAL) solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
main.py:98: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  figarray = table.hist(ax=ax, ylabelsize=5, xlabelsize=5)
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
[mlrun] 2020-03-10 12:31:57,786 log artifact summary at /User/demo-network-operations/artifacts/113baae308414b27ac6edc6034347bf8/summary.csv, size: None, db: Y
[mlrun] 2020-03-10 12:31:57,85

uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...347bf8,0,Mar 10 12:31:57,completed,sum,host=sum-24vdlkind=jobowner=admin,table,"class_labels=[0, 1]key=summarylabel_column=is_error",,summaryscale_pos_weightimbalancecorrelationhistograms


to track results use .show() or .logs() or in CLI: 
!mlrun get run 113baae308414b27ac6edc6034347bf8 --project network-operations , !mlrun logs 113baae308414b27ac6edc6034347bf8 --project network-operations
[mlrun] 2020-03-10 12:32:06,281 run executed, status=completed


## Save Project yaml

In [18]:
newproj.to_yaml(os.path.join(project_dir, 'project.yaml'))

TypeError: to_yaml() takes 1 positional argument but 2 were given